# 简介

[transitions](https://pypi.org/project/transitions/)是一个轻量级、面向对象的状态机库,还包括许多扩展.

# 安装

In [1]:
! pip install transitions

     ---------------------------------------- 0.0/96.7 kB ? eta -:--:--
     ------------ ------------------------- 30.7/96.7 kB 660.6 kB/s eta 0:00:01
     ---------------------------------------- 96.7/96.7 kB 1.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/111.8 kB ? eta -:--:--
   ---------------------------------------- 111.8/111.8 kB 3.3 MB/s eta 0:00:00


# 示例1: 视频处理

假设有一个视频类,它包括一些视频的基本信息

## 创建模型

In [11]:
from pydantic import BaseModel
from typing import Optional


class DouyinVideo(BaseModel):
    """
    抖音视频
    """
    id: str
    """视频id"""

    title: str
    """视频标题"""

    author: str
    """作者"""

    duration: int
    """视频时长"""

    url: str
    """视频链接"""

    download_url: Optional[str] = None
    """下载链接"""

    like_count: int = 0
    """点赞数量"""

    local_path: Optional[str] = None
    """本地路径"""

    play_url: Optional[str] = None
    """播放链接"""

    original_script: Optional[str] = None
    """原始文案"""

    rewritten_script: Optional[str] = None
    """改写后的文案"""

    digital_human_video_path: Optional[str] = None
    """数字人视频路径"""


## 创建状态机

现在需要为这个视频类创建一个状态机,包含以下几个状态:

1. 待下载
2. 待提取(文案)
3. 待洗稿
4. 待生成(数字人视频)

In [18]:
from transitions import Machine

class DouyinVideoStateMachine:
    states = ['待下载', '待提取', '待洗稿', '待生成', '完成']
    transitions = [
            { 'trigger': 'download', 'source': '待下载', 'dest': '待提取' },
            { 'trigger': 'extract', 'source': '待提取', 'dest': '待洗稿' },
            { 'trigger': 'rewrite', 'source': '待洗稿', 'dest': '待生成' },
            { 'trigger': 'generate', 'source': '待生成', 'dest': '完成' }
        ]
    state_actions = {
        '待下载': {'action': 'download_video', 'label': '下载视频'},
        '待提取': {'action': 'extract_original_script', 'label': '提取文案'},
        '待洗稿': {'action': 'rewrite_script', 'label': '洗稿'},
        '待生成': {'action': 'generate_digital_human_video', 'label': '生成数字人视频'},
    }

    def __init__(self, video: DouyinVideo):
        self.video = video
        self.machine = Machine(model=self, states=DouyinVideoStateMachine.states, transitions=DouyinVideoStateMachine.transitions, initial='待下载')
    def get_next_transition(self):
        current_state = self.state
        possible_transitions = self.machine.get_transitions(current_state)
        if possible_transitions:
            return possible_transitions[0]
        return None
    def download_video(self):
        print(f"下载视频: {self.video.title}")
        # 这里添加实际的下载逻辑
        self.video.local_path = f"/path/to/downloaded/{self.video.id}.mp4"
        self.download()

    def extract_original_script(self):
        print(f"从视频中提取文案: {self.video.title}")
        # 这里添加实际的文案提取逻辑
        self.video.original_script = f"原始{self.video.title}文案"
        self.extract()

    def rewrite_script(self):
        print(f"洗稿: {self.video.title}")
        # 这里添加实际的洗稿逻辑
        self.video.rewritten_script = f"洗稿后的{self.video.title}文案"
        self.rewrite()

    def generate_digital_human_video(self):
        print(f"生成数字人视频: {self.video.title}")
        # 这里添加实际的数字人视频生成逻辑
        self.video.digital_human_video_path = f"/path/to/digital_human/{self.video.id}.mp4"
        self.generate()

## 使用状态机

In [24]:
# 使用示例
video = DouyinVideo(
    id="123456",
    title="示例抖音视频",
    author="示例作者",
    duration=60,
    url="https://example.com/video/123456",
    download_url="https://example.com/download/123456"
)

state_machine = DouyinVideoStateMachine(video)

print(f"初始状态: {state_machine.state}")
print(f"下一个状态: {state_machine.get_next_transition()}")

state_machine.download_video()
print(f"下载后状态: {state_machine.state}")
print(f"本地路径: {video.local_path}")

state_machine.extract_original_script()
print(f"提取文案后状态: {state_machine.state}")
print(f"提取的文案: {video.original_script}")

state_machine.rewrite_script()
print(f"洗稿后状态: {state_machine.state}")
print(f"洗稿后的文案: {video.rewritten_script}")

state_machine.generate_digital_human_video()
print(f"生成数字人视频后状态: {state_machine.state}")
print(f"数字人视频路径: {video.digital_human_video_path}")

初始状态: 待下载
下一个状态: None
下载视频: 示例抖音视频
下载后状态: 待提取
本地路径: /path/to/downloaded/123456.mp4
从视频中提取文案: 示例抖音视频
提取文案后状态: 待洗稿
提取的文案: 原始示例抖音视频文案
洗稿: 示例抖音视频
洗稿后状态: 待生成
洗稿后的文案: 洗稿后的示例抖音视频文案
生成数字人视频: 示例抖音视频
生成数字人视频后状态: 完成
数字人视频路径: /path/to/digital_human/123456.mp4


# 核心概念

**状态(States)**:

* 系统在特定时刻的条件或阶段
* 上面例子中的`待下载`、`待提取`、`待洗稿`、`待生成`就是状态,分别代表视频处理过程中的一个特定阶段

**转换(Transitions)**:

* 从一个状态到另一个状态的过程或事件
* 上面例子中的`download_video`、`extract_original_script`、`rewrite_script`、`generate_digital_human_video`就是转换,它们用来将视频从一个状态转换到另一个状态